# Imports

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import h3
from shapely.geometry import Polygon, Point
import xarray as xr
import pandas as pd
import os

pd.set_option('display.max_columns', None)

# Reading in .nc files

In [2]:
def read_nc (file):
    ds = xr.open_dataset(file)
    df = ds.to_dataframe()
    df = df.reset_index()
    ds.close()
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
    gdf = gdf.set_crs(epsg=4326)
    gdf = gdf.dropna(subset=['wbgt'])
    gdf = gdf.drop(columns=['lat', 'lon', 'lev'])
    return gdf

In [3]:
# create a list of all files in unsynced/data/heat-risk
path = './data/heat-risk/'
heat_files = os.listdir(path)
heat_files = [f for f in heat_files if f.endswith('.nc')]
heat_files

['WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_hist_se6flor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp4_comflor-hist_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor-hist_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp8_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor-hist_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_hist_comflor-ERA5.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp8_comflor-hist_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_hist_se6flor-hist_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor-rcp8_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_hist_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_ERA5.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor-rcp4_comflor.nc',
 'WBGTid_COR-mdmx_AS_MesMax_rcp4_comflor.nc']

In [4]:
# using read_nc function to read in all files in heat_files and create a list of dataframes
heat_gdfs = [read_nc(path+f) for f in heat_files]
# heat_gdfs

In [5]:
# use gpd.sjoin to join the dataframes and add filename as string to add onto each column name
joined = gpd.sjoin(heat_gdfs[0], heat_gdfs[1], how='left', rsuffix=heat_files[1], lsuffix=None)
joined

,time,wbgt,geometry,index_WBGTid_COR-mdmx_AS_MesMax_hist_se6flor.nc,time_WBGTid_COR-mdmx_AS_MesMax_hist_se6flor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_hist_se6flor.nc
264,1987-01-01,30.183783,POINT (274.875 9.875),264,1987-01-01,26.619759
266,1987-01-01,30.084925,POINT (274.875 10.375),266,1987-01-01,26.387993
267,1987-01-01,30.314640,POINT (274.875 10.625),267,1987-01-01,26.687359
268,1987-01-01,30.544355,POINT (274.875 10.875),268,1987-01-01,26.986727
272,1987-01-01,31.463213,POINT (274.875 11.875),272,1987-01-01,28.184193
...,...,...,...,...,...,...
55800,1987-01-01,31.154692,POINT (324.875 -6.125),55800,1987-01-01,27.722589
56071,1987-01-01,30.393925,POINT (325.125 -7.875),56071,1987-01-01,27.334995
56072,1987-01-01,30.517490,POINT (325.125 -7.625),56072,1987-01-01,27.400921
56073,1987-01-01,30.641054,POINT (325.125 -7.375),56073,1987-01-01,27.466846


In [6]:
result = heat_gdfs[0]

result = heat_gdfs[0].copy()
result.columns = [f"{col}_{heat_files[0]}" if col != 'geometry' else col for col in result.columns]

for i in range(1, len(heat_gdfs)):
    right_gdf = heat_gdfs[i].copy()
    right_gdf.columns = [f"{col}_{heat_files[i]}" if col != 'geometry' else col for col in right_gdf.columns]
    
    joined = gpd.sjoin(result, right_gdf, how='left')
    joined = joined.drop(columns=['index_right'])
    
    # Remove _left and _right suffixes
    joined.columns = [col.replace('_left', '').replace('_right', '') for col in joined.columns]
    
    result = joined

result

,time_WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor.nc,geometry,time_WBGTid_COR-mdmx_AS_MesMax_hist_se6flor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_hist_se6flor.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp4_comflor-hist_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp4_comflor-hist_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor-hist_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor-hist_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp8_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp8_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor-hist_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor-hist_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_hist_comflor-ERA5.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_hist_comflor-ERA5.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp8_comflor-hist_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp8_comflor-hist_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_hist_se6flor-hist_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_hist_se6flor-hist_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor-rcp8_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp8_se6flor-rcp8_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_hist_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_hist_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor.nc,time_WBGTid_COR-mdmx_AS_MesMax_ERA5.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_ERA5.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor-rcp4_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp4_se6flor-rcp4_comflor.nc,time_WBGTid_COR-mdmx_AS_MesMax_rcp4_comflor.nc,wbgt_WBGTid_COR-mdmx_AS_MesMax_rcp4_comflor.nc
264,1987-01-01,30.183783,POINT (274.875 9.875),1987-01-01,26.619759,1987-01-01,27.923073,1987-01-01,28.047695,1987-01-01,29.901438,1987-01-01,30.183783,1987-01-01,26.531763,1987-01-01,29.901438,1987-01-01,26.619759,1987-01-01,30.183783,1987-01-01,26.531763,1987-01-01,28.047695,1987-01-01,26.383301,1987-01-01,28.047695,1987-01-01,27.923073
266,1987-01-01,30.084925,POINT (274.875 10.375),1987-01-01,26.387993,1987-01-01,27.704948,1987-01-01,27.858284,1987-01-01,29.761808,1987-01-01,30.084925,1987-01-01,26.281200,1987-01-01,29.761808,1987-01-01,26.387993,1987-01-01,30.084925,1987-01-01,26.281200,1987-01-01,27.858284,1987-01-01,26.232927,1987-01-01,27.858284,1987-01-01,27.704948
267,1987-01-01,30.314640,POINT (274.875 10.625),1987-01-01,26.687359,1987-01-01,27.991985,1987-01-01,28.145315,1987-01-01,30.008705,1987-01-01,30.314640,1987-01-01,26.593931,1987-01-01,30.008705,1987-01-01,26.687359,1987-01-01,30.314640,1987-01-01,26.593931,1987-01-01,28.145315,1987-01-01,26.552067,1987-01-01,28.145315,1987-01-01,27.991985
268,1987-01-01,30.544355,POINT (274.875 10.875),1987-01-01,26.986727,1987-01-01,28.279020,1987-01-01,28.432348,1987-01-01,30.255602,1987-01-01,30.544355,1987-01-01,26.906664,1987-01-01,30.255602,1987-01-01,26.986727,1987-01-01,30.544355,1987-01-01,26.906664,1987-01-01,28.432348,1987-01-01,26.871208,1987-01-01,28.432348,1987-01-01,28.279020
272,1987-01-01,31.463213,POINT (274.875 11.875),1987-01-01,28.184193,1987-01-01,29.427164,1987-01-01,29.580479,1987-01-01,31.243189,1987-01-01,31.463213,1987-01-01,28.157587,1987-01-01,31.243189,1987-01-01,28.184193,1987-01-01,31.463213,1987-01-01,28.157587,1987-01-01,29.580479,1987-01-01,28.147774,1987-01-01,29.580479,1987-01-01,29.427164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55800,1987-01-01,31.154692,POINT (324.875 -6.125),1987-01-01,27.722589,1987-01-01,29.138214,1987-01-01,29.083208,1987-01-01,31.105379,1987-01-01,31.154692,1987-01-01,27.751799,1987-01-01,31.105379,1987-01-01,27.722589,1987-01-01,31.154692,1987-01-01,27.751799,1987-01-01,29.083208,1987-01-01,27.563522,1987-01-01,29.083208,1987-01-01,29.138214
56071,1987-01-01,30.393925,POINT (325.125 -7.875),1987-01-01,27.334995,1987-01-01,28.663431,1987-01-01,28.532217,1987-01-01,30.355568,1987-01-01,30.393925,1987-01-01,27.357819,1987-01-01,30.355568,1987-01-01,27.334995,1987-

In [7]:
# drop any columns that include date in the name
result = result.loc[:,~result.columns.str.contains('time')]
# in the column names delete the string _WBGTid_COR-mdmx_AS_MesMax_
result.columns = result.columns.str.replace('WBGTid_COR-mdmx_AS_MesMax_', '')
result.columns = result.columns.str.replace('.nc', '')
result


,wbgt_rcp8_se6flor,geometry,wbgt_hist_se6flor,wbgt_rcp4_comflor-hist_comflor,wbgt_rcp4_se6flor-hist_comflor,wbgt_rcp8_comflor,wbgt_rcp8_se6flor-hist_comflor,wbgt_hist_comflor-ERA5,wbgt_rcp8_comflor-hist_comflor,wbgt_hist_se6flor-hist_comflor,wbgt_rcp8_se6flor-rcp8_comflor,wbgt_hist_comflor,wbgt_rcp4_se6flor,wbgt_ERA5,wbgt_rcp4_se6flor-rcp4_comflor,wbgt_rcp4_comflor
264,30.183783,POINT (274.875 9.875),26.619759,27.923073,28.047695,29.901438,30.183783,26.531763,29.901438,26.619759,30.183783,26.531763,28.047695,26.383301,28.047695,27.923073
266,30.084925,POINT (274.875 10.375),26.387993,27.704948,27.858284,29.761808,30.084925,26.281200,29.761808,26.387993,30.084925,26.281200,27.858284,26.232927,27.858284,27.704948
267,30.314640,POINT (274.875 10.625),26.687359,27.991985,28.145315,30.008705,30.314640,26.593931,30.008705,26.687359,30.314640,26.593931,28.145315,26.552067,28.145315,27.991985
268,30.544355,POINT (274.875 10.875),26.986727,28.279020,28.432348,30.255602,30.544355,26.906664,30.255602,26.986727,30.544355,26.906664,28.432348,26.871208,28.432348,28.279020
272,31.463213,POINT (274.875 11.875),28.184193,29.427164,29.580479,31.243189,31.463213,28.157587,31.243189,28.184193,31.463213,28.157587,29.580479,28.147774,29.580479,29.427164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55800,31.154692,POINT (324.875 -6.125),27.722589,29.138214,29.083208,31.105379,31.154692,27.751799,31.105379,27.722589,31.154692,27.751799,29.083208,27.563522,29.083208,29.138214
56071,30.393925,POINT (325.125 -7.875),27.334995,28.663431,28.532217,30.355568,30.393925,27.357819,30.355568,27.334995,30.393925,27.357819,28.532217,27.155945,28.532217,28.663431
56072,30.517490,POINT (325.125 -7.625),27.400921,28.742109,28.626717,30.475105,30.517490,27.424202,30.475105,27.400921,30.517490,27.424202,28.626717,27.234413,28.626717,28.742109
56073,30.641054,POINT (325.125 -7.375),27.466846,28.820787,28.721216,30.594643,30.641054,27.490587,30.594643,27.466846,30.641054,27.490587,28.721216,27.312881,28.721216,28.820787


# H3 joins

In [8]:
def point_to_h3(row, resolution):
    # Ensure we're using longitude first, then latitude
    return h3.latlng_to_cell(row.geometry.y, row.geometry.x, resolution)

def hexagon_to_polygon(h):
    # Get the boundaries in (lat, lng) order
    boundary = h3.cell_to_boundary(h)
    # Convert to (lng, lat) order for Shapely
    boundary = [(lng, lat) for lat, lng in boundary]
    return Polygon(boundary)

In [9]:
# Using gdf_heat_clean as the input GeoDataFrame
gdf_heat_clean = result
resolution = 4

if gdf_heat_clean.crs is None or not gdf_heat_clean.crs.is_geographic:
    gdf_heat_clean = gdf_heat_clean.to_crs("EPSG:4326")

# Convert points to H3 indexes
gdf_heat_clean['h3_index'] = gdf_heat_clean.apply(lambda row: point_to_h3(row, resolution), axis=1)

# Group by H3 index and calculate the mean of numeric columns
numeric_columns = gdf_heat_clean.select_dtypes(include=[np.number]).columns
h3_grouped = gdf_heat_clean.groupby('h3_index')[numeric_columns].mean().reset_index()

# Convert H3 indexes to polygons
h3_grouped['geometry'] = h3_grouped['h3_index'].apply(hexagon_to_polygon)

# Create a new GeoDataFrame with H3 hexagons
gdf_h3 = gpd.GeoDataFrame(h3_grouped, geometry='geometry', crs="EPSG:4326")

# gdf_h3.explore()

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


# Deforestation data

In [10]:
# read Dataset_butt_etal.csv as pandas dataframe
df_butt = pd.read_csv('./data/deforestation/Dataset_butt_etal.csv')
df_butt


,lat,lon,Latitude_rescale,Elevation_rescale,Distance_coast_rescale,local_0-2km_start,regional_2-5km_start,regional_5-10km_start,regional_10-25km_start,regional_25-50km_start,regional_50-100km_start,local_0-2km_end,regional_2-5km_end,regional_5-10km_end,regional_10-25km_end,regional_25-50km_end,regional_50-100km_end,Delta_T,regional_2-10km_start,regional_2-10km_end,regional_10-100km_start,regional_10-100km_end,train_test_split
0,2.825,-68.335,0.766579,0.242639,0.508327,0.985,0.99,0.99,0.99,0.97,0.96,0.985,0.99,0.99,0.99,0.97,0.95,-0.727778,0.990,0.990,0.973333,0.970000,train
1,-3.995,-48.025,0.542679,0.253007,0.159400,0.860,0.79,0.73,0.75,0.75,0.72,0.430,0.57,0.33,0.44,0.50,0.52,0.532636,0.760,0.450,0.740000,0.486667,train
2,-7.315,-55.255,0.433684,0.528292,0.527255,0.770,0.79,0.77,0.87,0.89,0.96,0.545,0.53,0.39,0.51,0.64,0.76,1.826663,0.780,0.460,0.906667,0.636667,train
3,-8.955,-54.245,0.379842,0.758576,0.582676,0.980,0.99,0.99,0.99,0.96,0.90,0.980,0.99,0.99,0.98,0.92,0.83,0.088342,0.990,0.990,0.950000,0.910000,train
4,-16.405,-60.475,0.135259,0.720493,0.706369,0.925,0.91,0.92,0.89,0.70,0.69,0.880,0.87,0.80,0.67,0.56,0.60,0.943321,0.915,0.835,0.760000,0.610000,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691626,-0.755,-55.755,0.649048,0.595535,0.296235,1.000,1.00,1.00,1.00,0.99,0.96,1.000,1.00,1.00,1.00,0.96,0.92,-0.187787,1.000,1.000,0.983333,0.960000,test
3691627,-5.655,-47.585,0.488181,0.339868,0.263421,0.185,0.50,0.45,0.38,0.41,0.44,0.130,0.37,0.31,0.27,0.29,0.30,0.793855,0.475,0.340,0.410000,0.286667,test
3691628,-0.125,-66.175,0.669731,0.153396,0.728065,0.995,0.99,0.98,0.94,0.96,0.98,0.995,0.99,0.98,0.94,0.95,0.97,-0.582327,0.985,0.985,0.960000,0.953333,test
3691629,-17.015,-57.795,0.115233,0.210958,0.871554,0.540,0.46,0.35,0.36,0.38,0.46,0.510,0.44,0.34,0.35,0.36,0.43,-0.338376,0.405,0.390,0.400000,0.380000,test


In [11]:
gdf_butt = gpd.GeoDataFrame(df_butt, geometry=gpd.points_from_xy(df_butt.lon, df_butt.lat))
gdf_butt = gdf_butt.set_crs(epsg=4326)
gdf_butt = gdf_butt.drop(columns=['lat', 'lon', "Latitude_rescale", "Elevation_rescale", "Distance_coast_rescale", "train_test_split"])

In [12]:
# Using gdf_heat_clean as the input GeoDataFrame
resolution = 4

if gdf_butt.crs is None or not gdf_butt.crs.is_geographic:
    gdf_butt = gdf_butt.to_crs("EPSG:4326")

# Convert points to H3 indexes
gdf_butt['h3_index'] = gdf_butt.apply(lambda row: point_to_h3(row, resolution), axis=1)

# Group by H3 index and calculate the mean of numeric columns
numeric_columns = gdf_butt.select_dtypes(include=[np.number]).columns
h3_grouped = gdf_butt.groupby('h3_index')[numeric_columns].mean().reset_index()

# Convert H3 indexes to polygons
h3_grouped['geometry'] = h3_grouped['h3_index'].apply(hexagon_to_polygon)

# Create a new GeoDataFrame with H3 hexagons
gdf_butt_h3 = gpd.GeoDataFrame(h3_grouped, geometry='geometry', crs="EPSG:4326")

# gdf_h3.explore()

In [23]:
# join gdf_butt_h3 with gdf_h3 on h3_index column dont do a spatial join
joined = gdf_h3.merge(gdf_butt_h3, on='h3_index', how='left',)

In [24]:
joined

,h3_index,wbgt_rcp8_se6flor,wbgt_hist_se6flor,wbgt_rcp4_comflor-hist_comflor,wbgt_rcp4_se6flor-hist_comflor,wbgt_rcp8_comflor,wbgt_rcp8_se6flor-hist_comflor,wbgt_hist_comflor-ERA5,wbgt_rcp8_comflor-hist_comflor,wbgt_hist_se6flor-hist_comflor,wbgt_rcp8_se6flor-rcp8_comflor,wbgt_hist_comflor,wbgt_rcp4_se6flor,wbgt_ERA5,wbgt_rcp4_se6flor-rcp4_comflor,wbgt_rcp4_comflor,geometry_x,local_0-2km_start,regional_2-5km_start,regional_5-10km_start,regional_10-25km_start,regional_25-50km_start,regional_50-100km_start,local_0-2km_end,regional_2-5km_end,regional_5-10km_end,regional_10-25km_end,regional_25-50km_end,regional_50-100km_end,Delta_T,regional_2-10km_start,regional_2-10km_end,regional_10-100km_start,regional_10-100km_end,geometry_y
0,845f001ffffffff,31.585396,29.242016,29.617168,30.134438,31.326138,31.585396,28.668076,31.326138,29.242016,31.585396,28.668076,30.134438,28.829144,30.134438,29.617168,"POLYGON ((-55.75823 3.59877, -55.56971 3.75876...",0.987550,0.987351,0.985364,0.989536,0.988675,0.983974,0.986887,0.985695,0.983642,0.986623,0.979934,0.974570,-0.459174,0.986358,0.984669,0.987395,0.980375,"POLYGON ((-55.75823 3.59877, -55.56971 3.75876..."
1,845f003ffffffff,31.443930,29.198391,29.705442,30.022825,31.458475,31.443930,28.776773,31.458475,29.198391,31.443930,28.776773,30.022825,28.949488,30.022825,29.705442,"POLYGON ((-56.01912 3.92071, -55.83105 4.08057...",0.990792,0.990545,0.990000,0.990000,0.990000,0.984505,0.990545,0.990446,0.990000,0.988960,0.985495,0.970842,-0.132953,0.990272,0.990223,0.988168,0.981766,"POLYGON ((-56.01912 3.92071, -55.83105 4.08057..."
2,845f005ffffffff,31.856033,29.386272,29.698301,30.358055,31.373306,31.856033,28.730766,31.373306,29.386272,31.856033,28.730766,30.358055,28.888756,30.358055,29.698301,"POLYGON ((-55.34429 3.67741, -55.15523 3.83758...",0.989821,0.988929,0.987976,0.986357,0.984595,0.960357,0.986357,0.985000,0.982643,0.978595,0.976119,0.945619,-0.096340,0.988452,0.983821,0.977103,0.966778,"POLYGON ((-55.34429 3.67741, -55.15523 3.83758..."
3,845f007ffffffff,31.620234,29.272282,29.768057,30.166447,31.489229,31.620234,28.822939,31.489229,29.272282,31.620234,28.822939,30.166447,28.991798,30.166447,29.768057,"POLYGON ((-55.60626 3.99982, -55.41764 4.15986...",0.988280,0.986453,0.985769,0.985321,0.981432,0.940598,0.963429,0.960085,0.958761,0.958162,0.962628,0.923098,-0.104107,0.986111,0.959423,0.969117,0.947963,"POLYGON ((-55.60626 3.99982, -55.41764 4.15986..."
4,845f009ffffffff,31.262779,29.008924,29.397949,29.859526,31.106407,31.262779,28.453413,31.106407,29.008924,31.262779,28.453413,29.859526,28.605125,29.859526,29.397949,"POLYGON ((-55.91018 3.1974, -55.72177 3.35731,...",0.985909,0.986234,0.986883,0.988442,0.990000,0.990000,0.985649,0.985844,0.985974,0.986364,0.987792,0.984675,-0.641985,0.986558,0.985909,0.989481,0.986277,"POLYGON ((-55.91018 3.1974, -55.72177 3.35731,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,84df6e5ffffffff,13.543766,10.884591,12.249022,11.819283,13.692493,13.543766,11.135985,13.692493,10.884591,13.543766,11.135985,11.819283,11.106785,11.819283,12.249022,"POLYGON ((-69.44647 -54.96638, -69.04143 -54.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
10344,84df6e7ffffffff,14.085174,11.495200,12.811290,12.384437,14.247598,14.085174,11.726002,14.247598,11.495200,14.085174,11.726002,12.384437,11.712502,12.384437,12.811290,"POLYGON ((-69.40015 -54.59002, -68.9999 -54.59...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
10345,84df6e9ffffffff,13.792105,11.486358,12.695211,12.322515,13.917169,13.792105,11.597439,13.917169,11.486358,13.792105,11.597439,12.322515,11.539921,12.322515,12.695211,"POLYGON ((-70.65806 -54.95698, -70.25483 -54.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
10346,84df6ebffffffff,14.102574,11.740191,12.958524,12.571511,14.233013,14.102574,11.874527,14.2330

In [28]:
# drop geometry_y and h3_index_y
# rename h3_index_x to h3_index and geometry_x to geometry
joined = joined.drop(columns=['geometry_y'])
joined = joined.rename(columns={'geometry_x': 'geometry'})
joined

,h3_index,wbgt_rcp8_se6flor,wbgt_hist_se6flor,wbgt_rcp4_comflor-hist_comflor,wbgt_rcp4_se6flor-hist_comflor,wbgt_rcp8_comflor,wbgt_rcp8_se6flor-hist_comflor,wbgt_hist_comflor-ERA5,wbgt_rcp8_comflor-hist_comflor,wbgt_hist_se6flor-hist_comflor,wbgt_rcp8_se6flor-rcp8_comflor,wbgt_hist_comflor,wbgt_rcp4_se6flor,wbgt_ERA5,wbgt_rcp4_se6flor-rcp4_comflor,wbgt_rcp4_comflor,geometry,local_0-2km_start,regional_2-5km_start,regional_5-10km_start,regional_10-25km_start,regional_25-50km_start,regional_50-100km_start,local_0-2km_end,regional_2-5km_end,regional_5-10km_end,regional_10-25km_end,regional_25-50km_end,regional_50-100km_end,Delta_T,regional_2-10km_start,regional_2-10km_end,regional_10-100km_start,regional_10-100km_end
0,845f001ffffffff,31.585396,29.242016,29.617168,30.134438,31.326138,31.585396,28.668076,31.326138,29.242016,31.585396,28.668076,30.134438,28.829144,30.134438,29.617168,"POLYGON ((-55.75823 3.59877, -55.56971 3.75876...",0.987550,0.987351,0.985364,0.989536,0.988675,0.983974,0.986887,0.985695,0.983642,0.986623,0.979934,0.974570,-0.459174,0.986358,0.984669,0.987395,0.980375
1,845f003ffffffff,31.443930,29.198391,29.705442,30.022825,31.458475,31.443930,28.776773,31.458475,29.198391,31.443930,28.776773,30.022825,28.949488,30.022825,29.705442,"POLYGON ((-56.01912 3.92071, -55.83105 4.08057...",0.990792,0.990545,0.990000,0.990000,0.990000,0.984505,0.990545,0.990446,0.990000,0.988960,0.985495,0.970842,-0.132953,0.990272,0.990223,0.988168,0.981766
2,845f005ffffffff,31.856033,29.386272,29.698301,30.358055,31.373306,31.856033,28.730766,31.373306,29.386272,31.856033,28.730766,30.358055,28.888756,30.358055,29.698301,"POLYGON ((-55.34429 3.67741, -55.15523 3.83758...",0.989821,0.988929,0.987976,0.986357,0.984595,0.960357,0.986357,0.985000,0.982643,0.978595,0.976119,0.945619,-0.096340,0.988452,0.983821,0.977103,0.966778
3,845f007ffffffff,31.620234,29.272282,29.768057,30.166447,31.489229,31.620234,28.822939,31.489229,29.272282,31.620234,28.822939,30.166447,28.991798,30.166447,29.768057,"POLYGON ((-55.60626 3.99982, -55.41764 4.15986...",0.988280,0.986453,0.985769,0.985321,0.981432,0.940598,0.963429,0.960085,0.958761,0.958162,0.962628,0.923098,-0.104107,0.986111,0.959423,0.969117,0.947963
4,845f009ffffffff,31.262779,29.008924,29.397949,29.859526,31.106407,31.262779,28.453413,31.106407,29.008924,31.262779,28.453413,29.859526,28.605125,29.859526,29.397949,"POLYGON ((-55.91018 3.1974, -55.72177 3.35731,...",0.985909,0.986234,0.986883,0.988442,0.990000,0.990000,0.985649,0.985844,0.985974,0.986364,0.987792,0.984675,-0.641985,0.986558,0.985909,0.989481,0.986277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,84df6e5ffffffff,13.543766,10.884591,12.249022,11.819283,13.692493,13.543766,11.135985,13.692493,10.884591,13.543766,11.135985,11.819283,11.106785,11.819283,12.249022,"POLYGON ((-69.44647 -54.96638, -69.04143 -54.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10344,84df6e7ffffffff,14.085174,11.495200,12.811290,12.384437,14.247598,14.085174,11.726002,14.247598,11.495200,14.085174,11.726002,12.384437,11.712502,12.384437,12.811290,"POLYGON ((-69.40015 -54.59002, -68.9999 -54.59...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10345,84df6e9ffffffff,13.792105,11.486358,12.695211,12.322515,13.917169,13.792105,11.597439,13.917169,11.486358,13.792105,11.597439,12.322515,11.539921,12.322515,12.695211,"POLYGON ((-70.65806 -54.95698, -70.25483 -54.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10346,84df6ebffffffff,14.102574,11.740191,12.958524,12.571511,14.233013,14.102574,11.874527,14.233013,11.740191,14.102574,11.874527,12.571511,11.846799,12.571511,12.958524,"POLYGON ((-70.59746 -54.58195, -70.19896 -54.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# set geometry to geometry
joined = joined.set_geometry('geometry')


In [26]:
# give an overview with max min averages of each column and write to a json file


In [44]:
def write_to_files(gdf):
    overview = gdf.describe()
    overview.to_json('heat_deforestation_data_overview.json', orient='index')
    gdf = gdf.fillna(0)
    gdf.to_file("heat_risk_deforestation.geojson", driver='GeoJSON')
# write_to_files(joined)

# Creating more descriptitve data

In [30]:
joined

,h3_index,wbgt_rcp8_se6flor,wbgt_hist_se6flor,wbgt_rcp4_comflor-hist_comflor,wbgt_rcp4_se6flor-hist_comflor,wbgt_rcp8_comflor,wbgt_rcp8_se6flor-hist_comflor,wbgt_hist_comflor-ERA5,wbgt_rcp8_comflor-hist_comflor,wbgt_hist_se6flor-hist_comflor,wbgt_rcp8_se6flor-rcp8_comflor,wbgt_hist_comflor,wbgt_rcp4_se6flor,wbgt_ERA5,wbgt_rcp4_se6flor-rcp4_comflor,wbgt_rcp4_comflor,geometry,local_0-2km_start,regional_2-5km_start,regional_5-10km_start,regional_10-25km_start,regional_25-50km_start,regional_50-100km_start,local_0-2km_end,regional_2-5km_end,regional_5-10km_end,regional_10-25km_end,regional_25-50km_end,regional_50-100km_end,Delta_T,regional_2-10km_start,regional_2-10km_end,regional_10-100km_start,regional_10-100km_end
0,845f001ffffffff,31.585396,29.242016,29.617168,30.134438,31.326138,31.585396,28.668076,31.326138,29.242016,31.585396,28.668076,30.134438,28.829144,30.134438,29.617168,"POLYGON ((-55.75823 3.59877, -55.56971 3.75876...",0.987550,0.987351,0.985364,0.989536,0.988675,0.983974,0.986887,0.985695,0.983642,0.986623,0.979934,0.974570,-0.459174,0.986358,0.984669,0.987395,0.980375
1,845f003ffffffff,31.443930,29.198391,29.705442,30.022825,31.458475,31.443930,28.776773,31.458475,29.198391,31.443930,28.776773,30.022825,28.949488,30.022825,29.705442,"POLYGON ((-56.01912 3.92071, -55.83105 4.08057...",0.990792,0.990545,0.990000,0.990000,0.990000,0.984505,0.990545,0.990446,0.990000,0.988960,0.985495,0.970842,-0.132953,0.990272,0.990223,0.988168,0.981766
2,845f005ffffffff,31.856033,29.386272,29.698301,30.358055,31.373306,31.856033,28.730766,31.373306,29.386272,31.856033,28.730766,30.358055,28.888756,30.358055,29.698301,"POLYGON ((-55.34429 3.67741, -55.15523 3.83758...",0.989821,0.988929,0.987976,0.986357,0.984595,0.960357,0.986357,0.985000,0.982643,0.978595,0.976119,0.945619,-0.096340,0.988452,0.983821,0.977103,0.966778
3,845f007ffffffff,31.620234,29.272282,29.768057,30.166447,31.489229,31.620234,28.822939,31.489229,29.272282,31.620234,28.822939,30.166447,28.991798,30.166447,29.768057,"POLYGON ((-55.60626 3.99982, -55.41764 4.15986...",0.988280,0.986453,0.985769,0.985321,0.981432,0.940598,0.963429,0.960085,0.958761,0.958162,0.962628,0.923098,-0.104107,0.986111,0.959423,0.969117,0.947963
4,845f009ffffffff,31.262779,29.008924,29.397949,29.859526,31.106407,31.262779,28.453413,31.106407,29.008924,31.262779,28.453413,29.859526,28.605125,29.859526,29.397949,"POLYGON ((-55.91018 3.1974, -55.72177 3.35731,...",0.985909,0.986234,0.986883,0.988442,0.990000,0.990000,0.985649,0.985844,0.985974,0.986364,0.987792,0.984675,-0.641985,0.986558,0.985909,0.989481,0.986277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,84df6e5ffffffff,13.543766,10.884591,12.249022,11.819283,13.692493,13.543766,11.135985,13.692493,10.884591,13.543766,11.135985,11.819283,11.106785,11.819283,12.249022,"POLYGON ((-69.44647 -54.96638, -69.04143 -54.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10344,84df6e7ffffffff,14.085174,11.495200,12.811290,12.384437,14.247598,14.085174,11.726002,14.247598,11.495200,14.085174,11.726002,12.384437,11.712502,12.384437,12.811290,"POLYGON ((-69.40015 -54.59002, -68.9999 -54.59...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10345,84df6e9ffffffff,13.792105,11.486358,12.695211,12.322515,13.917169,13.792105,11.597439,13.917169,11.486358,13.792105,11.597439,12.322515,11.539921,12.322515,12.695211,"POLYGON ((-70.65806 -54.95698, -70.25483 -54.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10346,84df6ebffffffff,14.102574,11.740191,12.958524,12.571511,14.233013,14.102574,11.874527,14.233013,11.740191,14.102574,11.874527,12.571511,11.846799,12.571511,12.958524,"POLYGON ((-70.59746 -54.58195, -70.19896 -54.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# for any column of join which includes the term km, substract the start and end values and create a delta column
delta_columns = []
start_list = []
end_list = []
for col in joined.columns:
    if 'km' in col:
        delta_columns.append(col)
for col in delta_columns:
    # take the text after the last _
    split_col_name = col.split('_')
    type = split_col_name[-1]
    if type == 'start':
        start_list.append(col)
    else:
        end_list.append(col)

for i in range(len(start_list)):
    start = start_list[i]
    end = end_list[i]
    delta = end.replace('end', 'delta')
    joined[delta] = joined[end] - joined[start]

joined

,h3_index,wbgt_rcp8_se6flor,wbgt_hist_se6flor,wbgt_rcp4_comflor-hist_comflor,wbgt_rcp4_se6flor-hist_comflor,wbgt_rcp8_comflor,wbgt_rcp8_se6flor-hist_comflor,wbgt_hist_comflor-ERA5,wbgt_rcp8_comflor-hist_comflor,wbgt_hist_se6flor-hist_comflor,wbgt_rcp8_se6flor-rcp8_comflor,wbgt_hist_comflor,wbgt_rcp4_se6flor,wbgt_ERA5,wbgt_rcp4_se6flor-rcp4_comflor,wbgt_rcp4_comflor,geometry,local_0-2km_start,regional_2-5km_start,regional_5-10km_start,regional_10-25km_start,regional_25-50km_start,regional_50-100km_start,local_0-2km_end,regional_2-5km_end,regional_5-10km_end,regional_10-25km_end,regional_25-50km_end,regional_50-100km_end,Delta_T,regional_2-10km_start,regional_2-10km_end,regional_10-100km_start,regional_10-100km_end,local_0-2km_delta,regional_2-5km_delta,regional_5-10km_delta,regional_10-25km_delta,regional_25-50km_delta,regional_50-100km_delta,regional_2-10km_delta,regional_10-100km_delta
0,845f001ffffffff,31.585396,29.242016,29.617168,30.134438,31.326138,31.585396,28.668076,31.326138,29.242016,31.585396,28.668076,30.134438,28.829144,30.134438,29.617168,"POLYGON ((-55.75823 3.59877, -55.56971 3.75876...",0.987550,0.987351,0.985364,0.989536,0.988675,0.983974,0.986887,0.985695,0.983642,0.986623,0.979934,0.974570,-0.459174,0.986358,0.984669,0.987395,0.980375,-0.000662,-0.001656,-0.001722,-0.002914,-0.008742,-0.009404,-0.001689,-0.007020
1,845f003ffffffff,31.443930,29.198391,29.705442,30.022825,31.458475,31.443930,28.776773,31.458475,29.198391,31.443930,28.776773,30.022825,28.949488,30.022825,29.705442,"POLYGON ((-56.01912 3.92071, -55.83105 4.08057...",0.990792,0.990545,0.990000,0.990000,0.990000,0.984505,0.990545,0.990446,0.990000,0.988960,0.985495,0.970842,-0.132953,0.990272,0.990223,0.988168,0.981766,-0.000248,-0.000099,0.000000,-0.001040,-0.004505,-0.013663,-0.000050,-0.006403
2,845f005ffffffff,31.856033,29.386272,29.698301,30.358055,31.373306,31.856033,28.730766,31.373306,29.386272,31.856033,28.730766,30.358055,28.888756,30.358055,29.698301,"POLYGON ((-55.34429 3.67741, -55.15523 3.83758...",0.989821,0.988929,0.987976,0.986357,0.984595,0.960357,0.986357,0.985000,0.982643,0.978595,0.976119,0.945619,-0.096340,0.988452,0.983821,0.977103,0.966778,-0.003464,-0.003929,-0.005333,-0.007762,-0.008476,-0.014738,-0.004631,-0.010325
3,845f007ffffffff,31.620234,29.272282,29.768057,30.166447,31.489229,31.620234,28.822939,31.489229,29.272282,31.620234,28.822939,30.166447,28.991798,30.166447,29.768057,"POLYGON ((-55.60626 3.99982, -55.41764 4.15986...",0.988280,0.986453,0.985769,0.985321,0.981432,0.940598,0.963429,0.960085,0.958761,0.958162,0.962628,0.923098,-0.104107,0.986111,0.959423,0.969117,0.947963,-0.024850,-0.026368,-0.027009,-0.027158,-0.018803,-0.017500,-0.026688,-0.021154
4,845f009ffffffff,31.262779,29.008924,29.397949,29.859526,31.106407,31.262779,28.453413,31.106407,29.008924,31.262779,28.453413,29.859526,28.605125,29.859526,29.397949,"POLYGON ((-55.91018 3.1974, -55.72177 3.35731,...",0.985909,0.986234,0.986883,0.988442,0.990000,0.990000,0.985649,0.985844,0.985974,0.986364,0.987792,0.984675,-0.641985,0.986558,0.985909,0.989481,0.986277,-0.000260,-0.000390,-0.000909,-0.002078,-0.002208,-0.005325,-0.000649,-0.003203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,84df6e5ffffffff,13.543766,10.884591,12.249022,11.819283,13.692493,13.543766,11.135985,13.692493,10.884591,13.543766,11.135985,11.819283,11.106785,11.819283,12.249022,"POLYGON ((-69.44647 -54.96638, -69.04143 -54.9...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10344,84df6e7ffffffff,14.085174,11.495200,12.811290,12.384437,14.247598,14.085174,11.726002,14.247598,11.495200,14.085174,11.726002,12.384437,11.712502,12.384437,12.811290,"POLYGON ((-69.40015 -54.59002, -68.9999 -54.59...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10345,84df6

In [45]:
write_to_files(joined)